# Train your first LM model with Amazon SageMaker
### Fine-tune a Multi-Class Classification with `Trainer` and `emotion` dataset and deploy it on a Sagemaker inference endpoint

# Introduction

Welcome to our end-to-end multi-class Text-Classification example. In this demo, we will use the Hugging Faces `transformers` and `datasets` library together with a custom Amazon sagemaker-sdk extension to fine-tune a pre-trained transformer for multi-class text classification. In particular, the pre-trained model will be fine-tuned using the `emotion` dataset. To get started, we need to set up the environment with a few prerequisite steps, for permissions, configurations, and so on. 


_**NOTE: You can run this demo in Sagemaker Studio, your local machine or Sagemaker Notebook Instances**_

# Development Environment and Permissions 

In [ ]:
!pip install "sagemaker>=2.140.0" "transformers==4.26.1" "datasets[s3]==2.10.1" --upgrade
!pip install "ipywidgets"

In [ ]:
import sagemaker.huggingface
import ipywidgets

## Permissions

In [ ]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

# Preprocessing

We are using the `datasets` library to download and preprocess the `emotion` dataset. After preprocessing, the dataset will be uploaded to our `sagemaker_session_bucket` to be used within our training job. The [emotion](https://github.com/dair-ai/emotion_dataset) dataset consists of 16000 training examples, 2000 validation examples, and 2000 testing examples.

## Tokenization 

In [ ]:
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer

# tokenizer used in preprocessing
tokenizer_name = 'distilbert-base-uncased'

# dataset used
dataset_name = 'emotion'

# s3 key prefix for the data
s3_prefix = 'samples/datasets/emotion'

In [ ]:
# download tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

# tokenizer helper function
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True)

# load dataset
train_dataset, test_dataset = load_dataset(dataset_name, split=['train', 'test'])

# tokenize dataset
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# set format for pytorch
train_dataset =  train_dataset.rename_column("label", "labels")
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset = test_dataset.rename_column("label", "labels")
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

The dataset includes the text column with the raw sentences we want the model to learn the "emotion", the labels that corresponds to coded emotions like sadness, anger, surprise, fear and so on. Column input_ids correspond to the tokenized version of the sentence and finally the attention_mask, used by the masked self attention layer.

In [ ]:
train_dataset.to_pandas().head(10)

## Uploading data to `sagemaker_session_bucket`

After we processed the `datasets` we are going to use the new `FileSystem` [integration](https://huggingface.co/docs/datasets/filesystems.html) to upload our dataset to S3.

In [ ]:
# save train_dataset to s3
training_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/train'
train_dataset.save_to_disk(training_input_path)

# save test_dataset to s3
test_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/test'
test_dataset.save_to_disk(test_input_path)

## Creating an Estimator and start a training job

From all supported models from Hugging Face, we selected https://huggingface.co/distilbert-base-uncased. It is light ("distilled") version of the BERT model, introduced in [this paper](https://arxiv.org/abs/1910.01108), where authors claim to have reduced the size of the model by 40% compared to BERT-base, while retaining 97%
of its language understanding capabilities and being 60% faster. Size reduction and faster execution helps to reduce the footprint of the instance required for training and inference, reducing trainig time.

### Language Model Training (Model Fine-Tuning)
Language Models size ranges from hundreds of million to hundreds of billion of parameters. For example BERT-base consists of 110M parameters. As training such models can take weeks, months ore much more, it is common practise to start from a pre-trained model and tune the network parameters (called model fine-tuning) using domain specific datasets with supervised learning. 

This is what we will do in the following. As you will see, the training (fine-tuning) on the emotion data set, takes just about 12 minutes. 


In [ ]:
from sagemaker.huggingface import HuggingFace
from huggingface_hub import HfFolder
import time

# hyperparameters, which are passed into the training job
hyperparameters={'epochs': 1,                                    # number of training epochs
                 'train_batch_size': 32,                         # batch size for training
                 'eval_batch_size': 64,                          # batch size for evaluation
                 'learning_rate': 3e-5,                          # learning rate used during training
                 'model_id':'distilbert-base-uncased',           # pre-trained model
                 'fp16': True,                                   # Whether to use 16-bit (mixed) precision training
                }

In [ ]:
# define Training Job Name 
job_name = f'Train-First-LM-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'

# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'train.py',        # fine-tuning script used in training jon
    source_dir           = './scripts',       # directory where fine-tuning script is stored
    instance_type        = 'ml.p3.2xlarge',   # instances type used for the training job
    instance_count       = 1,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    transformers_version = '4.26',           # the transformers version used in the training job
    pytorch_version      = '1.13',           # the pytorch_version version used in the training job
    py_version           = 'py39',            # the python version used in the training job
    hyperparameters      = hyperparameters,   # the hyperparameter used for running the training job
)

In [ ]:
# define a data input dictonary with our uploaded s3 uris
data = {
    'train': training_input_path,
    'test': test_input_path
}

# starting the train job with our uploaded datasets as input
# setting wait to False to not expose the HF Token
huggingface_estimator.fit(data, wait=False)

In [ ]:
# adding waiter to see when training is done
waiter = huggingface_estimator.sagemaker_session.sagemaker_client.get_waiter('training_job_completed_or_stopped')
waiter.wait(TrainingJobName=huggingface_estimator.latest_training_job.name)

While waiting for training to complete, which will take about 12 minutes, you can move to the SageMaker Console, click on Training -> Training Jobs and you will see your training job (InProgress Status) that you started calling the fit method in the cell above.  

When the training is completed you can see your training Job in Completed Status. Clicking on the job name you are directed to a page reporting several details of your training job, in particular at bottom of the page you can find the location of the model artifact you just created. 

## Deploying the model to a SageMaker Endpoint

To deploy our model to Amazon SageMaker we use the the `hugginface_estimator` to deploy our model from S3 with `huggingface_estimator.deploy()`. This operation takes a few minutes.

In [ ]:
predictor = huggingface_estimator.deploy(
    initial_instance_count=1, 
    instance_type='ml.m5.xlarge')


While waiting for the endpoint to be created, you can go back to the Sagemaket console, click on Inference and then endpoint to check the status of the inference endpoint creation. When the status is In Service, we can the returned predictor object to call the endpoint and get sentiment prediction.

In [ ]:
sentiment_input= {"inputs": "Winter is coming and it will be dark soon."}

predictor.predict(sentiment_input)

Finally, we delete the inference endpoint.

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()